# Hyperparameter Optimization (Powerspec Model)

In [1]:
import logging
import random
import os
import time
import datetime
import warnings
from itertools import product

import torch
from torch import nn, optim
from torch.optim.lr_scheduler import ExponentialLR
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import h5py
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split, KFold

from utils import load_mel_data, count_parameters
from models import *

warnings.filterwarnings("ignore")

# Make code deterministic
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Config and global variables

In [2]:
# Set file paths
BASE_PATH = os.path.join("/home/source/experiments/")
RESULTS_BASE_PATH = os.path.join(BASE_PATH, "results")
LOG_FILE_PATH = os.path.join(BASE_PATH, "exp03_train.log")
MODEL_BASE_PATH = os.path.join(BASE_PATH, 'exp03_models')
TRAIN_RESULTS_PATH = os.path.join(RESULTS_BASE_PATH, "exp03_train.csv")
TEST_RESULTS_PATH = os.path.join(RESULTS_BASE_PATH, "exp03_test.csv")

# Logging config
logging.basicConfig(level=logging.INFO,
                    filename=LOG_FILE_PATH,
                    format='%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

# Set information about the dataset
HDF5_FILE_PATH = os.path.join(os.sep, 'home', 'data', "ANNOTATED_BEDTIME_TU7.hdf5")
COLNAMES = ["Time", "X", "Y", "Z", "Annotated Time in Bed"]
SAMPLE_RATE = 100
LABEL_DICT = {False: 0, True: 1}
EXCLUDED_DATASETS = []

# Set information about the model, etc.
INPUT_DIM = 160
OUTPUT_DIM = 1

DROPOUT = 0.5 # https://jmlr.org/papers/v15/srivastava14a.html
BATCH_SIZE = 8
CLIP = 1000
MAX_EPOCHS = 256
MIN_EPOCHS = 0
LR_DECAY = .9
REVERSE = False

# Combinations to test
MODELS = [MLP, RNN, LSTM]
HID_DIM = [1,2,4,8,16,32,64]
N_LAYERS = [1,2,4]
INIT_LR = [.7]

# Minimal required loss impprovement
EPSILON = 1e-4

means = torch.Tensor([ -48.4244,  -65.1251,  -71.5329,  -76.1376,  -79.9719,  -83.3733,
                       -86.4463,  -89.2009,  -91.6243,  -93.1006,  -95.0616,  -97.5081,
                       -98.9459, -100.2535, -101.4587, -101.9269, -103.0128, -104.7198,
                      -105.6010, -105.6779, -106.4788, -107.9100, -108.5404, -108.4596,
                      -109.0225, -110.2787, -110.0530, -110.5040, -111.6329, -111.2709,
                      -111.6229, -111.9671, -112.2292, -113.2221, -112.7077, -112.8696,
                      -113.0403, -113.1380, -113.2175, -113.2942,  -48.2297,  -66.3363,
                       -73.2646,  -77.7978,  -81.1189,  -84.1860,  -86.7041,  -88.9725,
                       -90.9699,  -92.1340,  -93.9985,  -96.3562,  -97.8235,  -99.1135,
                      -100.1286, -100.4643, -101.3244, -103.0018, -103.9505, -104.0375,
                      -104.8962, -106.1595, -106.6999, -106.5312, -106.8978, -108.3458,
                      -108.1114, -108.5288, -109.7681, -109.1479, -109.5088, -109.7915,
                      -109.8525, -111.1021, -110.4775, -110.5919, -110.9078, -110.7910,
                      -110.9218, -111.0520,  -43.2504,  -61.6844,  -68.9458,  -73.5961,
                       -77.0992,  -80.0608,  -82.7051,  -85.1344,  -87.3067,  -88.6262,
                       -90.4330,  -92.7584,  -94.2015,  -95.5068,  -96.7073,  -97.2398,
                       -98.3422, -100.0732, -100.9963, -101.1373, -101.9556, -103.4150,
                      -104.0725, -104.0774, -104.7053, -106.0052, -105.8673, -106.3338,
                      -107.4961, -107.1521, -107.5086, -107.8810, -108.1648, -109.1684,
                      -108.6960, -108.9002, -109.0938, -109.1790, -109.2591, -109.3175,
                       -88.4886,  -95.3217,  -97.4304,  -98.6252,  -99.8187, -101.9569,
                      -103.8922, -105.9184, -107.8997, -109.0467, -110.9091, -113.2237,
                      -114.7497, -116.0618, -117.1553, -117.7753, -118.7548, -120.5024,
                      -121.3765, -121.2271, -122.1441, -123.5200, -124.1884, -124.2784,
                      -124.6106, -126.1395, -125.7348, -126.0794, -127.6051, -126.9591,
                      -127.5139, -127.9155, -127.7043, -129.1599, -128.4229, -128.5986,
                      -129.2023, -129.0043, -129.1702, -129.2966])

stds = torch.Tensor([15.2323, 16.0104, 15.9714, 16.1902, 15.9361, 15.5933, 15.1956, 14.7352,
                     14.3270, 13.9598, 13.6914, 13.4495, 13.2140, 13.0092, 12.8071, 12.6164,
                     12.4418, 12.3007, 12.1658, 11.9811, 11.9132, 11.8334, 11.7669, 11.7018,
                     11.6241, 11.6008, 11.5206, 11.4817, 11.4794, 11.4014, 11.3947, 11.3801,
                     11.3306, 11.3842, 11.3314, 11.3115, 11.3362, 11.3181, 11.2947, 11.3140,
                     18.7676, 19.9467, 19.3003, 19.1615, 18.9609, 18.9122, 18.3342, 17.8444,
                     17.2975, 16.7270, 16.3592, 15.8516, 15.6143, 15.3346, 14.9757, 14.8446,
                     14.5318, 14.3890, 14.2145, 13.8998, 13.8854, 13.6931, 13.6441, 13.6196,
                     13.3293, 13.4485, 13.1998, 13.0278, 13.2340, 12.9852, 13.0790, 13.0648,
                     12.7679, 13.0264, 12.7902, 12.7130, 12.9849, 12.7771, 12.8141, 12.8975,
                     12.6477, 13.1166, 12.7965, 12.9296, 13.0482, 13.1621, 13.1321, 12.9195,
                     12.6833, 12.4614, 12.2735, 12.1418, 12.0261, 11.9286, 11.8221, 11.7176,
                     11.5904, 11.5109, 11.4179, 11.3001, 11.2326, 11.1805, 11.1244, 11.0692,
                     11.0040, 10.9983, 10.9474, 10.9163, 10.9429, 10.8745, 10.8632, 10.8572,
                     10.8032, 10.8757, 10.8285, 10.8419, 10.8991, 10.8676, 10.8668, 10.8800,
                     22.3097, 25.5176, 24.0877, 23.6057, 23.0063, 22.4626, 21.4491, 20.4060,
                     19.4148, 18.4030, 17.5354, 16.7295, 16.0956, 15.5101, 14.9024, 14.3493,
                     13.8040, 13.3513, 12.9347, 12.4549, 12.1130, 11.7816, 11.4755, 11.1741,
                     10.8611, 10.6525, 10.3602, 10.1416,  9.9928,  9.7345,  9.5809,  9.4059,
                      9.2305,  9.1791,  9.0191,  8.9312,  8.8625,  8.7818,  8.7032,  8.6692])

## Load data

In [3]:
def load_dataset(file_path, subjects, label_dict, resampled_frequency="1min", means=None, stds=None):

    X, y = zip(*[load_mel_data(file_path, subject, label_dict, sample_rate=SAMPLE_RATE, resampled_frequency=resampled_frequency, colnames=COLNAMES) for subject in tqdm(subjects, desc="Loading data")])

    lengths = [elem.shape[0] for elem in X]

    X, y, lengths = zip(*[(X[ii], y[ii], lengths[ii]) for ii in np.argsort(lengths)[::-1]])
    
    means, stds = torch.cat(X).mean(axis=0), torch.cat(X).std(axis=0)
    
    logging.info(f"means = {means}; stds = {stds}")
    print(f"means = {means}; stds = {stds}")

    class_0, class_1 = zip(*[((elem == 0).sum().numpy()/elem.shape[0], (elem == 1).sum().numpy()/elem.shape[0]) for elem in y])
    logging.info(f"Class 0 (awake): {np.mean(class_0):.2f} +/- {np.std(class_0):.2f}; Class 1 (sleep): {np.mean(class_1):.2f} +/- {np.std(class_1):.2f}")
    print(f"Class 0 (awake): {np.mean(class_0):.2f} +/- {np.std(class_0):.2f}; Class 1 (sleep): {np.mean(class_1):.2f} +/- {np.std(class_1):.2f}")

    X, y, lengths = pad_sequence(X, batch_first=True), pad_sequence(y, batch_first=True), torch.Tensor(lengths)

    if means is not None and stds is not None:
        X = (X - means) / stds
        logging.info("Normalized the input of each channel")
        print("Normalized the input of each channel")

    return X, y, lengths


# Select device (GPU if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load available subjects
with h5py.File(HDF5_FILE_PATH) as hdf5_file:
    subjects = [subject for subject in hdf5_file.keys() if subject not in EXCLUDED_DATASETS]

# Load the data
X, y, lengths = load_dataset(HDF5_FILE_PATH, subjects, LABEL_DICT, means=means, stds=stds)
X, y = X.float(), y.float()
X, y, lengths = X.to(device), y.to(device), lengths.to(device)
assert X.shape[0] == y.shape[0]
print(f"Loaded {X.shape[0]} sequences with input shape [{X.shape[1]} x {X.shape[2]}] and output shape [{y.shape[1]}]\n")

Loading data:   0%|          | 0/445 [00:00<?, ?it/s]

means = tensor([ -48.4244,  -65.1251,  -71.5329,  -76.1376,  -79.9719,  -83.3733,
         -86.4463,  -89.2009,  -91.6243,  -93.1006,  -95.0616,  -97.5081,
         -98.9459, -100.2535, -101.4587, -101.9269, -103.0128, -104.7198,
        -105.6010, -105.6779, -106.4788, -107.9100, -108.5404, -108.4596,
        -109.0225, -110.2787, -110.0530, -110.5040, -111.6329, -111.2709,
        -111.6229, -111.9671, -112.2292, -113.2221, -112.7077, -112.8696,
        -113.0403, -113.1380, -113.2175, -113.2942,  -48.2297,  -66.3363,
         -73.2646,  -77.7978,  -81.1189,  -84.1860,  -86.7041,  -88.9725,
         -90.9699,  -92.1340,  -93.9985,  -96.3562,  -97.8235,  -99.1135,
        -100.1286, -100.4643, -101.3244, -103.0018, -103.9505, -104.0375,
        -104.8962, -106.1595, -106.6999, -106.5312, -106.8978, -108.3458,
        -108.1114, -108.5288, -109.7681, -109.1479, -109.5088, -109.7915,
        -109.8525, -111.1021, -110.4775, -110.5919, -110.9078, -110.7910,
        -110.9218, -111.0520, 

## Create result files

In [4]:
with open(TRAIN_RESULTS_PATH, "w") as f:
    f.write("Combination,Fold,Epoch,Train Loss,Validation Loss,Hidden Dimension,Number of Layers,Initial Learning Rate,Model\n")
logging.info(f"Created training result file at {TRAIN_RESULTS_PATH}")

with open(TEST_RESULTS_PATH, "w") as f:
    f.write("Combination,Fold,Loss,Accuracy,Precision,Recall,F1 Score,Hidden Dimension,Number of Layers,Initial Learning Rate,Model,Ellapsed Time\n")
logging.info(f"Created test result file at {TEST_RESULTS_PATH}")

## Train the models

In [5]:
combinations = [(0, INIT_LR[0], 0, GLM)] + list(product(N_LAYERS, INIT_LR, HID_DIM, MODELS))
n_combinations = len(combinations)
for combination, (n_layers, init_lr, hid_dim, model_constr) in enumerate(tqdm(combinations)):

    logging.info(f"Combination {combination}: hid_dim = {hid_dim}; n_layers = {n_layers}; init_lr = {init_lr}; device = {device}")

    # Do 10-fold cross-validation
    kf = KFold(n_splits=10)
    for fold, (train_idx, test_idx) in enumerate(kf.split(np.arange(X.size(0)))):

        # Create validation data
        train_idx, valid_idx = train_test_split(np.arange(train_idx.shape[0]), test_size=0.2)

        # Create model and init weights
        model = model_constr(INPUT_DIM, hid_dim, OUTPUT_DIM, n_layers, dropout=DROPOUT, batch_first=True)
        logging.info('Model initialized with %s trainable parameters' % count_parameters(model))

        # Init loss and optimizer
        optimizer = optim.SGD(model.parameters(), lr=init_lr) # https://arxiv.org/abs/1409.3215
        scheduler = ExponentialLR(optimizer, gamma=LR_DECAY)
        criterion = nn.BCELoss()
        logging.info(f"Start with learning rate = {init_lr} (decay = {LR_DECAY}); batch size = {BATCH_SIZE}.")

        # Create dataloaders
        train_loader = DataLoader(TensorDataset(X[train_idx], y[train_idx], lengths[train_idx]), batch_size=BATCH_SIZE, shuffle=True)
        valid_loader = DataLoader(TensorDataset(X[valid_idx], y[valid_idx], lengths[valid_idx]), batch_size=BATCH_SIZE)
        test_loader = DataLoader(TensorDataset(X[test_idx], y[test_idx], lengths[test_idx]), batch_size=BATCH_SIZE)
        logging.info(f"Use {len(train_idx)} sequences for training, {len(valid_idx)} sequences for validation and {len(test_idx)} sequences for testing.")

        # Set path and init best loss
        best_model_path = os.path.join(MODEL_BASE_PATH, f'{combination:02d}_best_{n_layers}l_{model.name}{hid_dim}_model_fold_{fold}.pt')
        best_valid_loss = float('inf')
        epoch = 0

        overall_start_time = time.time()

        # Evaluate model without any training
        train_loss, _ = evaluate(model, train_loader, criterion)
        valid_loss, _ = evaluate(model, valid_loader, criterion)

        # Save losses to result file
        with open(TRAIN_RESULTS_PATH, "a") as f:
            f.write(f"{combination},{fold},{epoch},{train_loss},{valid_loss},{hid_dim},{n_layers},{init_lr},{model.name}\n")

        for epoch in range(1, MAX_EPOCHS + 1):

            start_time = time.time()

            train_loss = train(model, train_loader, optimizer, criterion, CLIP)
            valid_loss, _ = evaluate(model, valid_loader, criterion)

            time_diff = int(time.time() - start_time)

            scheduler.step()

            if valid_loss + EPSILON < best_valid_loss:
                # Save losses to result file
                with open(TRAIN_RESULTS_PATH, "a") as f:
                    f.write(f"{combination},{fold},{epoch},{train_loss},{valid_loss},{hid_dim},{n_layers},{init_lr},{model.name}\n")

                # Update best validation loss and save model
                best_valid_loss = valid_loss
                logging.info(f"Updated best validation loss to {best_valid_loss}.")
                torch.save(model.state_dict(), best_model_path)
            else:
                logging.info(f"End training after epoch {epoch} as validation loss does not further decrease.")
                logging.info(f"Best model saved at {best_model_path}")
                break

        time_diff = int(time.time() - overall_start_time)

        # Evaluate model on test set
        logging.info(f"Load model from epoch {epoch-1} from {best_model_path}")
        model.load_state_dict(torch.load(best_model_path))

        test_loss, metrics = evaluate(model, test_loader, criterion)
        accuracy, precision, recall, f1_score = metrics

        with open(TEST_RESULTS_PATH, "a") as f:
            f.write(f"{combination},{fold},{test_loss},{accuracy},{precision},{recall},{f1_score},{hid_dim},{n_layers},{init_lr},{model.name},{time_diff}\n")

  0%|          | 0/64 [00:00<?, ?it/s]